In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import os

# Set paths
train_dir = r"C:/Users/Nanda Kisore Biswas/Desktop/BE/DL_Datasets/flowers/train"
test_dir = r"C:/Users/Nanda Kisore Biswas/Desktop/BE/DL_Datasets/flowers/test"

# Image dimensions
img_height = 224
img_width = 224
batch_size = 32

# Define ImageDataGenerators for training and testing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'  # Since we have 5 categories
)

validation_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

# Load MobileNetV2 pre-trained on ImageNet without the top layer (classification head)
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Freeze the base model so that its weights are not updated during training
base_model.trainable = False

# Create a custom head for our model
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1024, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(5, activation='softmax')  # 5 classes for flowers
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Print model summary
model.summary()
# Define callbacks
checkpoint = ModelCheckpoint('flower_model.keras', monitor='val_accuracy', save_best_only=True, mode='max')
early_stop = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=2,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    callbacks=[checkpoint, early_stop]
)
# Unfreeze the base model
base_model.trainable = True

# Re-compile the model with a lower learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Continue training
history_fine = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=2,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    callbacks=[checkpoint, early_stop]
)
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(validation_generator, steps=validation_generator.samples // batch_size)
print(f'Test Accuracy: {test_acc * 100:.2f}%')
import numpy as np
from tensorflow.keras.preprocessing import image

# Load an image
img_path = r"C:/Users/Nanda Kisore Biswas/Desktop/BE/DL_Datasets/flowers/test/dandelion/dandelion_141.jpg"
img = image.load_img(img_path, target_size=(img_height, img_width))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
img_array /= 255.0  # Normalize the image

# Predict the class
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions, axis=1)

# Map predicted class to flower category
class_labels = list(train_generator.class_indices.keys())
predicted_label = class_labels[predicted_class[0]]

print(f'Predicted class: {predicted_label}')

Found 3250 images belonging to 5 classes.
Found 530 images belonging to 5 classes.


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1024)           │     1,311,744 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │         5,125 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,574,853 (13.64 MB)

 Trainable params: 1,316,869 (5.02 MB)

...

Epoch 1/2
101/101 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.6598 - loss: 1.0689

C:\Users\Nanda Kisore Biswas\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


101/101 ━━━━━━━━━━━━━━━━━━━━ 385s 4s/step - accuracy: 0.6609 - loss: 1.0651 - val_accuracy: 0.8516 - val_loss: 0.4229
Epoch 2/2
  1/101 ━━━━━━━━━━━━━━━━━━━━ 1:22 830ms/step - accuracy: 0.6875 - loss: 0.5819

C:\Users\Nanda Kisore Biswas\anaconda3\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


101/101 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.6875 - loss: 0.5819 - val_accuracy: 0.8333 - val_loss: 0.4191
Epoch 1/2
101/101 ━━━━━━━━━━━━━━━━━━━━ 871s 7s/step - accuracy: 0.7241 - loss: 0.7484 - val_accuracy: 0.8730 - val_loss: 0.3549
Epoch 2/2
101/101 ━━━━━━━━━━━━━━━━━━━━ 10s 47ms/step - accuracy: 0.8750 - loss: 0.3073 - val_accuracy: 0.9444 - val_loss: 0.2115
16/16 ━━━━━━━━━━━━━━━━━━━━ 16s 960ms/step - accuracy: 0.8700 - loss: 0.3523
Test Accuracy: 87.70%
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
Predicted class: dandelion


In [ ]:
import matplotlib.pyplot as plt

# Get class labels
class_names = list(test_data.class_indices.keys())

# Select a batch of images from the test set
images, true_labels = next(test_data)  # Get a batch of images and labels
predicted_probs = model.predict(images)
predicted_labels = np.argmax(predicted_probs, axis=1)
true_labels = np.argmax(true_labels, axis=1)

# Display a few images with predictions and actual labels
num_images = 5  # Number of images to display
plt.figure(figsize=(15, 10))
for i in range(num_images):
    plt.subplot(1, num_images, i + 1)
    plt.imshow(images[i])
    plt.axis('off')
    true_label = class_names[true_labels[i]]
    predicted_label = class_names[predicted_labels[i]]
    plt.title(f"True: {true_label}\nPred: {predicted_label}", color=color)

plt.show()
